In [23]:
import os
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import altair as alt

#### Loading and preparing data

In [2]:
# don't run this every time! re-import the exported csv to save time.
# commented out below to prevent accidental execution.

# dirs = [("DevGPT/" + d) for d in os.listdir("DevGPT") if os.path.isdir("DevGPT/" + d)] # modify this to limit snapshots

# files = []
# for dir in dirs:
#     for f in os.listdir(dir):
#         if f[-5:] == ".json": files.append(dir + "/" + f)

# print(files)

# imports = []
# for f in files:
#     if f[-5:] == ".json":  # this one pulls all files
#         trydf = pd.read_json(f)
#         trydf = pd.json_normalize(trydf["Sources"])
#         trydf["filepath"] = f
#         imports.append(trydf)

# df = pd.concat(imports, ignore_index = True)
# df.to_csv("df-export.csv", index = False)

['DevGPT/snapshot_20230727/20230727_195816_hn_sharings.json', 'DevGPT/snapshot_20230727/20230727_195927_pr_sharings.json', 'DevGPT/snapshot_20230727/20230727_195941_issue_sharings.json', 'DevGPT/snapshot_20230727/20230727_195954_discussion_sharings.json', 'DevGPT/snapshot_20230727/20230727_200003_commit_sharings.json', 'DevGPT/snapshot_20230727/20230727_200102_file_sharings.json', 'DevGPT/snapshot_20230803/20230803_093947_pr_sharings.json', 'DevGPT/snapshot_20230803/20230803_094705_issue_sharings.json', 'DevGPT/snapshot_20230803/20230803_094811_discussion_sharings.json', 'DevGPT/snapshot_20230803/20230803_095317_commit_sharings.json', 'DevGPT/snapshot_20230803/20230803_103605_file_sharings.json', 'DevGPT/snapshot_20230803/20230803_105332_hn_sharings.json', 'DevGPT/snapshot_20230810/20230810_123110_pr_sharings.json', 'DevGPT/snapshot_20230810/20230810_123938_issue_sharings.json', 'DevGPT/snapshot_20230810/20230810_124048_discussion_sharings.json', 'DevGPT/snapshot_20230810/20230810_1248

In [3]:
dfi = pd.read_csv("df-export.csv")

C:\Users\seamu\AppData\Local\Temp\ipykernel_7784\848057934.py:1: DtypeWarning: Columns (5,7,17,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  dfi = pd.read_csv("df-export.csv")


In [4]:
dfa = dfi[["Type", "Author", "Title", "Body", "Message", "CommitMessage", "filepath"]]

# query statements, checking which columns have text to extract

hn_q = "filepath.str.contains('hn_sharings.json')"     # Title
pr_q = "filepath.str.contains('pr_sharings.json')"     # Title, Body
is_q = "filepath.str.contains('issue_sharings.json')"     # Title, Body
di_q = "filepath.str.contains('discussion_sharings.json')"    # Title, Body
co_q = "filepath.str.contains('commit_sharings.json')"         # Message
fi_q = "filepath.str.contains('file_sharings.json')"          # CommitMessage

for query in [hn_q, pr_q, is_q, di_q, co_q, fi_q]:
    print(~dfa.query(query)[["Title", "Body", "Message", "CommitMessage"]].head(1).isna())

   Title   Body  Message  CommitMessage
0   True  False    False          False
     Title  Body  Message  CommitMessage
138   True  True    False          False
     Title  Body  Message  CommitMessage
285   True  True    False          False
     Title  Body  Message  CommitMessage
520   True  True    False          False
     Title   Body  Message  CommitMessage
552  False  False     True          False
     Title   Body  Message  CommitMessage
731  False  False    False           True


In [5]:
# combine text columns

dfa["text"] = dfa[["Title", "Body", "Message", "CommitMessage"]].agg(lambda x: "".join(x.dropna().astype(str)), axis = 1)
dfa = dfa.drop(["Title", "Body", "Message", "CommitMessage", "filepath"], axis = 1)

dfa

C:\Users\seamu\AppData\Local\Temp\ipykernel_7784\3042202609.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfa["text"] = dfa[["Title", "Body", "Message", "CommitMessage"]].agg(lambda x: "".join(x.dropna().astype(str)), axis = 1)


,Type,Author,text
0,hacker news,cbowal,OpenAI shuts down its AI Classifier due to poo...
1,hacker news,warrenm,“Devil’s horsemen”: Why Mongol horse archers w...
2,hacker news,ayhanfuat,The Fall of Stack Overflow
3,hacker news,cdme,Death Metal English (2013)
4,hacker news,notRobot,Shopify employee breaks NDA to reveal firm rep...
...,...,...,...
18709,discussion,l1t1,how to install on proot ubuntu of termux```\r\...
18710,discussion,rattrayalex,v3 to v4 Migration Guidev4 is a complete rewri...
18711,discussion,lukecookssw,"2023-06-9 - Databases, data warehouses, and da..."
18712,discussion,ZoeLeBlanc,AI & CLI AssignmentPost the results of your as...


In [6]:
dfa["text_pp"] = dfa["text"].apply(lambda x: gensim.utils.simple_preprocess(x))
dfa = dfa.drop(["text"], axis = 1)

dfa

,Type,Author,text_pp
0,hacker news,cbowal,"[openai, shuts, down, its, ai, classifier, due..."
1,hacker news,warrenm,"[devil, horsemen, why, mongol, horse, archers,..."
2,hacker news,ayhanfuat,"[the, fall, of, stack, overflow]"
3,hacker news,cdme,"[death, metal, english]"
4,hacker news,notRobot,"[shopify, employee, breaks, nda, to, reveal, f..."
...,...,...,...
18709,discussion,l1t1,"[how, to, install, on, proot, ubuntu, of, term..."
18710,discussion,rattrayalex,"[to, migration, guidev, is, complete, rewrite,..."
18711,discussion,lukecookssw,"[databases, data, warehouses, and, data, lakes..."
18712,discussion,ZoeLeBlanc,"[ai, cli, assignmentpost, the, results, of, yo..."


In [17]:
dfa["Author"].nunique()

1883

#### Apply model - basic

In [7]:
# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    dfa["text_pp"], dfa["Author"] , test_size = 0.2, random_state = 2)

# apply word vector model on training data
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size = 100,
                                   window = 5,
                                   min_count = 2)

words = set(w2v_model.wv.index_to_key)

# create average sentence vectors to train on
X_train_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                         if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                         for ls in X_train])

X_test_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                         if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                         for ls in X_test])

# define random forest model
rf = RandomForestClassifier(n_estimators = 100, random_state = 2)

# fit rf model to vectorized training data
rf.fit(X_train_vect, y_train)

# predict values on vectorized test data
y_pred = rf.predict(X_test_vect)

# report accuracy of test data predictions
print("Test set classification accuracy: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Test set classification accuracy: 86.91 %


#### Apply model - optimized

In [8]:
# checking how many rows are removed for minimum comment thresholds
for i in [1, 2, 5, 10, 100]:
    count = len(dfa.groupby("Author").filter(lambda x: len(x) >= i))
    perc = count / len(dfa) * 100
    print("%i rows in dataframe for authors with %i or more submissions -- %.1f %% of original." %
          (count, i, perc))

18714 rows in dataframe for authors with 1 or more submissions -- 100.0 % of original.
18320 rows in dataframe for authors with 2 or more submissions -- 97.9 % of original.
17217 rows in dataframe for authors with 5 or more submissions -- 92.0 % of original.
10553 rows in dataframe for authors with 10 or more submissions -- 56.4 % of original.
5146 rows in dataframe for authors with 100 or more submissions -- 27.5 % of original.


In [9]:
# remove text from authors that have fewer than 5 comments
dfb = dfa.groupby("Author").filter(lambda x: len(x) >= 5)

# include post type as a token 
dfb["text_pp"] = dfb.apply(lambda row: row["text_pp"] + [row["Type"]], axis = 1)

dfb

,Type,Author,text_pp
0,hacker news,cbowal,"[openai, shuts, down, its, ai, classifier, due..."
1,hacker news,warrenm,"[devil, horsemen, why, mongol, horse, archers,..."
2,hacker news,ayhanfuat,"[the, fall, of, stack, overflow, hacker news]"
3,hacker news,cdme,"[death, metal, english, hacker news]"
4,hacker news,notRobot,"[shopify, employee, breaks, nda, to, reveal, f..."
...,...,...,...
18708,discussion,kyzer-davis,"[name, for, uuid, with, all, bits, set, to, ff..."
18709,discussion,l1t1,"[how, to, install, on, proot, ubuntu, of, term..."
18710,discussion,rattrayalex,"[to, migration, guidev, is, complete, rewrite,..."
18711,discussion,lukecookssw,"[databases, data, warehouses, and, data, lakes..."


In [18]:
dfb["Author"].nunique()

1105

##### Tuning w2v_model parameters:

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    dfb["text_pp"], dfb["Author"] , test_size = 0.2, random_state = 2) 
# included Type column in text column this time for additional predictive power

skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 2)

for vec, win in [[50, 5], [50, 10], [50, 15],
                 [100, 5], [100, 10], [100, 15],
                 [200, 5], [200, 10], [200, 15]]:

    scores = []
    for train_index, test_index in skf.split(X_train, y_train):

        X_tr = X_train.iloc[train_index]
        X_te = X_train.iloc[test_index]
        y_tr = y_train.iloc[train_index]
        y_te = y_train.iloc[test_index]

        w2v_model = gensim.models.Word2Vec(X_tr,
                                        vector_size = vec,
                                        window = win,
                                        min_count = 1)
        # not too much extra to include all words: 12394 for count 1, 11685 for count 2 (default)

        words = set(w2v_model.wv.index_to_key)

        X_train_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                                if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                                for ls in X_tr])

        X_test_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                                if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                                for ls in X_te])

        rf = RandomForestClassifier(n_estimators = 100, random_state = 2)

        # fit rf model to vectorized training data
        rf.fit(X_train_vect, y_tr)

        # predict values on vectorized test data
        y_pred = rf.predict(X_test_vect)

        scores.append(accuracy_score(y_te, y_pred))
    # report accuracy of test data predictions
    print("For model with vector size %i and window size %i:\nClassification accuracy: %.3f , std dev: %.3f" % (vec, win, np.mean(scores), np.std(scores)))

C:\Users\seamu\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


For model with vector size 50 and window size 5:
Classification accuracy: 0.918 , std dev: 0.003


C:\Users\seamu\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


For model with vector size 50 and window size 10:
Classification accuracy: 0.917 , std dev: 0.004


C:\Users\seamu\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


...

For model with vector size 200 and window size 5:
Classification accuracy: 0.918 , std dev: 0.004


C:\Users\seamu\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


For model with vector size 200 and window size 10:
Classification accuracy: 0.918 , std dev: 0.004


C:\Users\seamu\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


For model with vector size 200 and window size 15:
Classification accuracy: 0.918 , std dev: 0.004


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    dfb["text_pp"], dfb["Author"] , test_size = 0.2, random_state = 2) 

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size = 100,
                                   window = 10,
                                   min_count = 1)

words = set(w2v_model.wv.index_to_key)

X_train_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                         if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                         for ls in X_train])

X_test_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                         if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                         for ls in X_test])

# define possible hyperparameters to tune
rf_params = {"criterion": ["gini", "entropy", "log_loss"],
             "max_features": ["sqrt", "log2", None],
             "n_estimators": [10, 20, 50, 100],
           "max_depth": range(1, 20)}

rf_tuned = RandomizedSearchCV(RandomForestClassifier(), rf_params, n_iter = 10, cv = skf,
                               random_state = 2, verbose = 1)

rf_tuned.fit(X_train_vect, y_train)

y_pred = rf_tuned.predict(X_test_vect)

print("Test set classification accuracy: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\seamu\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Test set classification accuracy: 92.33 %


In [12]:
print(rf_tuned.best_score_)
rf_tuned.best_params_

0.9182454822353758


{'n_estimators': 100,
 'max_features': 'sqrt',
 'max_depth': 13,
 'criterion': 'log_loss'}

In [13]:
# try to predict category?

X_train, X_test, y_train, y_test = train_test_split(
    dfa["text_pp"], dfa["Type"] , test_size = 0.2, random_state = 2) 

w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size = 50,
                                   window = 10,
                                   min_count = 1)

words = set(w2v_model.wv.index_to_key)

X_train_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                         if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                         for ls in X_train])

X_test_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                         if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                         for ls in X_test])

# define possible hyperparameters to tune
rf_params = {"criterion": ["gini", "entropy", "log_loss"],
             "max_features": ["sqrt", "log2", None],
             "n_estimators": [10, 20, 50, 100],
           "max_depth": range(1, 20)}

rf_tuned = RandomizedSearchCV(RandomForestClassifier(), rf_params, n_iter = 10, cv = skf,
                               random_state = 2, verbose = 1)

rf_tuned.fit(X_train_vect, y_train)

y_pred = rf_tuned.predict(X_test_vect)

print("Test set classification accuracy: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Test set classification accuracy: 97.60 %


In [ ]:
minposts = []
accuracies = []

for min_posts in range(1, 101, 2):
    dfc = dfa.groupby("Author").filter(lambda x: len(x) >= min_posts)
    dfc["text_pp"] = dfc.apply(lambda row: row["text_pp"] + [row["Type"]], axis = 1)

    X_train, X_test, y_train, y_test = train_test_split(
    dfc["text_pp"], dfc["Author"] , test_size = 0.2, random_state = 2) 

    w2v_model = gensim.models.Word2Vec(X_train,
                                    vector_size = 100,
                                    window = 10,
                                    min_count = 1)

    words = set(w2v_model.wv.index_to_key)

    X_train_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                            if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                            for ls in X_train])

    X_test_vect = np.array([np.mean([w2v_model.wv[i] for i in ls if i in words], axis=0) 
                            if any(i in words for i in ls) else np.zeros(w2v_model.vector_size)
                            for ls in X_test])

    rfc = RandomForestClassifier(n_estimators = 100, criterion = "log_loss", max_depth = 13, max_features = "sqrt")

    rfc.fit(X_train_vect, y_train)

    y_pred = rfc.predict(X_test_vect)

    minposts.append(min_posts)
    accuracies.append(accuracy_score(y_test, y_pred) * 100)

    # print("Test set classification accuracy for users with %i or more posts: %.2f %%" % (min_posts, accuracy_score(y_test, y_pred) * 100))

Test set classification accuracy for users with 1 or more posts: 87.15 %
Test set classification accuracy for users with 3 or more posts: 90.69 %
Test set classification accuracy for users with 5 or more posts: 92.36 %
Test set classification accuracy for users with 7 or more posts: 92.91 %
Test set classification accuracy for users with 9 or more posts: 94.23 %
Test set classification accuracy for users with 11 or more posts: 95.12 %
Test set classification accuracy for users with 13 or more posts: 95.67 %
Test set classification accuracy for users with 15 or more posts: 96.17 %
Test set classification accuracy for users with 17 or more posts: 96.11 %
Test set classification accuracy for users with 19 or more posts: 97.16 %
Test set classification accuracy for users with 21 or more posts: 97.98 %
Test set classification accuracy for users with 23 or more posts: 97.46 %
Test set classification accuracy for users with 25 or more posts: 98.19 %
Test set classification accuracy for users 

In [78]:
dfg = pd.DataFrame(columns = ["Minimum posts", "Accuracy"])
dfg["Minimum posts"] = minposts
dfg["Accuracy"] = accuracies

base = alt.Chart(dfg).mark_point(fill = "black", stroke = "black").encode(
    x = alt.X("Minimum posts"),
    y = alt.Y("Accuracy", title = "Accuracy (%)").scale(zero = False)
).properties(
    width = 800,
    height = 200
)

alt.layer(base.mark_line(color = "grey"), base).configure_axis(titleFontSize = 16, labelFontSize = 16)

alt.LayerChart(...)